# Fine Tuning a Lasagne Neural Network

In [1]:
%matplotlib inline
import matplotlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
#from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import log_loss
from nolearn.lasagne import NeuralNet
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.updates import adagrad, nesterov_momentum, adadelta, adam
from lasagne.nonlinearities import softmax, sigmoid, identity

from sklearn.preprocessing import StandardScaler
from pandas.tools.plotting import scatter_matrix

from lasagne.nonlinearities import sigmoid, softmax
from nolearn.lasagne.base import TrainSplit, BatchIterator

from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from project import utils


C:\Anaconda3\envs\env_name34\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
C:\Anaconda3\envs\env_name34\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Import Data

In [2]:
X, X_test, y = utils.load_data('log_flipped_trunc')

#test_x = pd.read_csv('x_test.csv', sep=';')
#X =     np.log(1 + X)
#X_all = np.log(1 + X_all)

X_all = pd.concat((X_test, X), axis=0)
scaler = StandardScaler()
scaler.fit(X_all)
X = scaler.transform(X)

encoder = LabelEncoder()
y = encoder.fit_transform(y).astype(np.int32)
num_classes = len(encoder.classes_)
X = np.array(X).astype(np.float32)
num_features = X.shape[1]

In [3]:
space = {
#'choice':
#     hp.choice('num_layers',
#     [
#                     {'layers':'two',

#                     },
#                     {'layers':'three',
                      
                      
#                       'units3': hp.choice('units3', [64, 128, 256, 512, 1024]),
#                       'dropout3': hp.choice('dropout3', [0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.2, 0.3, 0.4, 0.5, 0.65, 0.8])
                                
#                     }
        
    
#     ]),
    
    'units1': hp.choice('units1', [64, 128, 256, 512]),
    'units2': hp.choice('units2', [64, 128, 256, 512]),
    

    #'dropout_in': hp.choice('dropout_in', [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.1]),
    'dropout1': hp.choice('dropout1', [0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.2]),
    'dropout2': hp.choice('dropout2', [0.4, 0.5, 0.65, 0.8, 0.85, 0.9]),
    
    'batch_size' : hp.choice('batch_size', [128, 256]),

    'nb_epochs' :  50,
    #'update': hp.choice('update', [adagrad, nesterov_momentum, adadelta, adam]),
    'update': nesterov_momentum,
    'momentum': 0.9,#hp.choice('momentum', [0.7, 0.8, 0.9]),
    'nolin_2': hp.choice('nolin_2', [sigmoid, identity]),
    'learning_rate': 0.01,
         
}

In [ ]:
best_params = None
best_score = 1000000
kf = StratifiedKFold(n_splits=5, random_state=1205, shuffle=True)

def objective(params):
    global best_params, best_score
    print ('\n ')
    print ('Params testing: ', params)
    
    layers = [('input', InputLayer),
              #('dropout_in', DropoutLayer),
              ('dense1', DenseLayer),
              ('dropout1', DropoutLayer),
              ('dense2', DenseLayer),
              ('dropout2', DropoutLayer),
             ]
    
#     if params['choice']['layers']== 'three':
#         layers.append(('dense3', DenseLayer))
#         layers.append(('dropout3', DropoutLayer))
    
    layers.append(('output', DenseLayer))
    
    nn_params = dict(
                 layers=layers,
                 input_shape=(None, num_features),
                 
                 #dropout_in_p=params['dropout_in'],
                 
                 dense1_num_units=params['units1'],
                 dropout1_p=params['dropout1'],
                 
                 dense2_num_units=params['units2'],
                 dropout2_p=params['dropout2'],
                 dense2_nonlinearity=params['nolin_2'],
                 
                 batch_iterator_train=BatchIterator(batch_size=params['batch_size']),
                 batch_iterator_test=BatchIterator(batch_size=params['batch_size']),
                 
                 output_num_units=num_classes,
                 output_nonlinearity=softmax,
                 
                 update=params['update'],

                 update_learning_rate=params['learning_rate'],
                 update_momentum=params['momentum'],

                 train_split=TrainSplit(eval_size=0.2),
                 verbose=0,
                 max_epochs=params['nb_epochs']
    )
    
#     if params['choice']['layers']== 'three':
#         nn_params['dropout3_p'] = params['choice']['dropout3']
#         nn_params['dense3_num_units'] = params['choice']['units3']

    net = NeuralNet(**nn_params)
    scores = cross_val_score(net, X, y, cv=kf, scoring='neg_log_loss', n_jobs=1)
    score2 = -np.mean(scores)
    std = np.std(scores)
    
    score = round(score2, 4) * 10000 + std
    
    if score < best_score:
        best_score = score
        best_params = params
        print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        print('New best params with score:', score)
    
    print("Score:", score2)
    print("Std:", std)
 
    return {'loss': score, 'status': STATUS_OK}

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
    
trials = Trials()

best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=1000)

print('FINISHED!================')
print (best)


 
Params testing:  {'dropout2': 0.5, 'units2': 512, 'nb_epochs': 50, 'learning_rate': 0.01, 'momentum': 0.9, 'batch_size': 256, 'update': <function nesterov_momentum at 0x000000000B45C598>, 'dropout1': 0.025, 'units1': 64, 'nolin_2': <function linear at 0x000000000B2EC2F0>}
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
New best params with score: 3842.00496435
Score: 0.384207442163
Std: 0.00496434688439

 
Params testing:  {'dropout2': 0.65, 'units2': 512, 'nb_epochs': 50, 'learning_rate': 0.01, 'momentum': 0.9, 'batch_size': 256, 'update': <function nesterov_momentum at 0x000000000B45C598>, 'dropout1': 0.01, 'units1': 64, 'nolin_2': <function linear at 0x000000000B2EC2F0>}
Score: 0.384295892915
Std: 0.00514529672849

 
Params testing:  {'dropout2': 0.5, 'units2': 256, 'nb_epochs': 50, 'learning_rate': 0.01, 'momentum': 0.9, 'batch_size': 256, 'update': <function nesterov_momentum at 0x000000000B45C598>, 'dropout1': 0, 'units1': 64, 'nolin_2': <function sigmoid at 0x000000000B2EA84

In [ ]:
trials.best_trial

In [ ]:
# Params testing:  {'dropout2': 0.85, 'units2': 256, 'nb_epochs': 50, 'learning_rate': 0.01, 'momentum': 0.9, 'batch_size': 128, 'update': <function nesterov_momentum at 0x000000000B45C598>, 'dropout1': 0.01, 'units1': 128, 'nolin_2': <function linear at 0x000000000B2EC2F0>}
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# New best params with score: 3832.00536331